In [ ]:
!pip install yfinance

In [ ]:
from sklearn.decomposition import PCA
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


In [20]:
from data_query import fetch_NASDAQ100_index
from data_query import fetch_NASDAQ100_all_components
from data_query import fetch_SP500_index
from data_query import fetch_SP500_all_components
from data_query import fetch_ftse100_index
from data_query import fetch_ftse100_all_components
from PCA_function import rolling_pca_weights
from preprocessing import preprocessing_X
from sklearn.decomposition import PCA
from typing import List

In [3]:
import sys
import os
sys.path.append(os.path.abspath("../app"))
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/home/sb/lewagon_london/project_lewagon/stock-stat-replica/data/lewagon-statistical-arbitrage-ae470f7dcd48.json'


In [19]:
# from main import compute_bt_result
# bt_result,rep_pf=compute_bt_result(30,30,30,30,4,[0.5, 2, -0.5, -2],'SP500')


In [ ]:
index_selected='FTSE100'
if index_selected=='NASDAQ100':
    target_df= fetch_NASDAQ100_index()
    underlying_df=fetch_NASDAQ100_all_components()
elif index_selected=='SP500':
    target_df= fetch_SP500_index()
    underlying_df=fetch_SP500_all_components()
elif index_selected=='FTSE100':
    target_df= fetch_ftse100_index()
    underlying_df=fetch_ftse100_all_components()

n_stocks = 30               # number of stocks used for the replication
window = 30                 # period the trading strat goes
n_pcs = 3   
if 'date' in underlying_df.columns and not underlying_df['date'].empty:
    underlying_df.set_index('date', inplace=True)

# Create log returns to remove stationarity
log_returns = np.log(underlying_df / underlying_df.shift(1))

# Drop NaN values caused by the shift
log_returns = log_returns.dropna()
X_log = log_returns.copy()
stock_log_features = X_log.columns

# Scaling data
scaler = StandardScaler()
scaler.fit(X_log)
X_log = pd.DataFrame(scaler.transform(X_log), columns=stock_log_features, index=log_returns.index)

rep_pf = rolling_pca_weights(X_log, n_stocks, window, n_pcs)

In [37]:
daily_weights_df=rep_pf.copy()
daily_weights_df

,AAF,AAL,ABF,ADM,AHT,ALW,ANTO,AUTO,AV,AZN,...,SVT,TSCO,TW,ULVR,UTG,UU,VOD,WEIR,WPP,WTB
date,,,,,,,,,,,,,,,,,,,,,
2022-03-16,0.000000,0.0,0.000000,0.034438,0.035694,0.025299,0.000000,0.031097,0.000000,0.000000,...,0.000000,0.000000,0.025374,0.000000,0.025995,0.000000,0.000000,0.053511,0.000000,0.000000
2022-03-17,0.000000,0.0,0.038887,0.000000,0.000000,0.000000,0.000000,0.000000,0.032751,0.000000,...,0.000000,0.000000,0.000000,0.031526,0.000000,0.000000,0.000000,0.000000,0.032779,0.020963
2022-03-18,0.000000,0.0,0.039764,0.000000,0.000000,0.000000,0.000000,0.000000,0.032266,0.000000,...,0.000000,0.000000,0.000000,0.028702,0.000000,0.000000,0.000000,0.000000,0.033973,0.021611
2022-03-21,0.000000,0.0,0.039276,0.000000,0.000000,0.000000,0.000000,0.000000,0.032757,0.000000,...,0.000000,0.000000,0.000000,0.028750,0.000000,0.000000,0.000000,0.000000,0.034111,0.021394
2022-03-22,0.035390,0.0,0.000000,0.043168,0.000000,0.000000,0.000000,0.000000,0.027078,0.029024,...,0.000000,0.000000,0.000000,0.023753,0.025603,0.000000,0.000000,0.056879,0.038497,0.020763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-04,0.025841,0.0,0.026043,0.000000,0.029796,0.034330,0.000000,0.000000,0.028601,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028196,0.000000,0.000000
2025-03-05,0.000000,0.0,0.000000,0.000000,0.030581,0.032144,0.000000,0.000000,0.027354,0.000000,...,0.000000,0.000000,0.000000,0.034242,0.000000,0.000000,0.029973,0.028977,0.000000,0.024865
2025-03-06,0.040618,0.0,0.031321,0.000000,0.000000,0.037978,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033347,0.028945,0.032982,0.000000,0.031892,0.000000,0.000000,0.040038,0.000000


In [38]:
daily_weights_df.index=pd.to_datetime(daily_weights_df.index)
daily_weights_df.index


DatetimeIndex(['2022-03-16', '2022-03-17', '2022-03-18', '2022-03-21',
               '2022-03-22', '2022-03-23', '2022-03-24', '2022-03-25',
               '2022-03-28', '2022-03-29',
               ...
               '2025-02-25', '2025-02-26', '2025-02-27', '2025-02-28',
               '2025-03-03', '2025-03-04', '2025-03-05', '2025-03-06',
               '2025-03-07', '2025-03-10'],
              dtype='datetime64[ns]', name='date', length=755, freq=None)

In [41]:
target_df.set_index('date',inplace=True)

In [43]:
df=target_df

In [46]:
# SPREAD CODE:

pca_date = '2024-03-18'

# Filter stocks with values > 0 on the specific date
filtered_columns = daily_weights_df.loc[pca_date][daily_weights_df.loc[pca_date] > 0].index
# replication pf weights taken from output function
rep_pf = daily_weights_df.loc[[pca_date], filtered_columns]

# log returns of rep pf
rep_pf_log_returns_daily = log_returns[rep_pf.columns]

# log return times weights (results)
rep_pf_results = rep_pf_log_returns_daily.mul(rep_pf.iloc[0], axis=1)
rep_pf_results["total_rep_pf"] = rep_pf_results.sum(axis=1)
rep_pf_results

# Calculate log returns FTSE100 / Index    
FTSE_log_return = np.log(df["FTSE100"] / df["FTSE100"].shift(1)).dropna()
FTSE_log_return

# DF of spread
spread_df = pd.DataFrame()  # Create a new DataFrame for the spread
spread_df["spread"] = FTSE_log_return - rep_pf_results["total_rep_pf"]  # Add a column named "spread"
training_spread_df = spread_df.loc[:pca_date].iloc[:-1]  # Data up to the PCA date, excluding the date itself

In [48]:
training_spread_df

,spread
date,
2022-02-02,-0.001915
2022-02-03,0.005052
2022-02-04,0.009731
2022-02-07,0.005202
2022-02-08,-0.002083
...,...
2024-03-11,0.00242
2024-03-12,-0.002849
2024-03-13,0.005767
